In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kumarmanoj-bag-of-words-meets-bags-of-popcorn/testData.tsv
/kaggle/input/kumarmanoj-bag-of-words-meets-bags-of-popcorn/labeledTrainData.tsv
/kaggle/input/kumarmanoj-bag-of-words-meets-bags-of-popcorn/sampleSubmission.csv
/kaggle/input/kumarmanoj-bag-of-words-meets-bags-of-popcorn/unlabeledTrainData.tsv


In [2]:
!nvidia-smi

Mon Jun  2 07:46:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             25W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!pip install bitsandbytes wandb evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
!pip install evaluate

In [ ]:
import os
import pandas as pd
import numpy as np
import datasets
import evaluate
import torch
import wandb

from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.model_selection import train_test_split

os.environ["WANDB_API_KEY"] = ""

def load_data(path):
    return pd.read_csv(path, sep='\t', encoding='utf-8')

train = load_data("/kaggle/input/kumarmanoj-bag-of-words-meets-bags-of-popcorn/labeledTrainData.tsv")
test = load_data("/kaggle/input/kumarmanoj-bag-of-words-meets-bags-of-popcorn/testData.tsv")

train, val = train_test_split(train, test_size=0.2, random_state=42)
train_dataset = datasets.Dataset.from_dict({'label': train["sentiment"], 'text': train['review']})
val_dataset = datasets.Dataset.from_dict({'label': val["sentiment"], 'text': val['review']})
test_dataset = datasets.Dataset.from_dict({"text": test['review']})

model_id = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, max_length=512)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

alpha_list = [8, 16, 32]
r_list = [4, 8, 16]
target_module_options = [
    ["query_proj"]
]

for alpha in alpha_list:
    for r in r_list:
        for target_modules in target_module_options:
            exp_name = f"alpha_{alpha}_r_{r}_target_{'_'.join(target_modules)}"
            wandb.init(
                project="lora-distilbert-experiment",
                name=exp_name,
                config={
                    "model": model_id,
                    "lora_alpha": alpha,
                    "lora_r": r,
                    "target_modules": target_modules,
                    "epochs": 3,
                    "batch_size": 64
                }
            )

            model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
            lora_config = LoraConfig(
                r=r,
                lora_alpha=alpha,
                target_modules=target_modules,
                lora_dropout=0.05,
                bias="none",
                task_type=TaskType.SEQ_CLS
            )
            model = get_peft_model(model, lora_config)

            training_args = TrainingArguments(
                output_dir=f"./output/{exp_name}",
                num_train_epochs=3,
                per_device_train_batch_size=16,
                per_device_eval_batch_size=16,
                gradient_accumulation_steps=4, 
                warmup_steps=50,
                weight_decay=0.01,
                logging_dir="./",
                logging_steps=25,
                save_strategy="no",
                report_to="wandb",
                fp16=True,
                ddp_find_unused_parameters=False
            )

            trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=tokenized_train,
                eval_dataset=tokenized_val,
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_metrics,
            )

            trainer.train()
            eval_results = trainer.evaluate()
            wandb.log({"validation_accuracy": eval_results["eval_accuracy"]})

            prediction_outputs = trainer.predict(tokenized_test)
            test_pred = np.argmax(prediction_outputs[0], axis=-1).flatten()
            result_output = pd.DataFrame(data={"id": test["id"], "sentiment": test_pred})
            output_file = f"distilbert_lora_{exp_name}.csv"
            result_output.to_csv(output_file, index=False, quoting=3)

            wandb.finish()


2025-06-02 07:48:08.461969: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748850488.671599      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748850488.728346      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


...

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:           eval/accuracy ▁
wandb:               eval/loss ▁
wandb:            eval/runtime ▁
wandb: eval/samples_per_second ▁
wandb:   eval/steps_per_second ▁
wandb:            test/runtime ▁
wandb: test/samples_per_second ▁
wandb:   test/steps_per_second ▁
wandb:             train/epoch ▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
wandb:       train/global_step ▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇██████
wandb:         train/grad_norm ▃▃▂▂▁▁▂▃▃▂▂▄▃▂▅▃▅▆▅▄▅▄▄▃▄▄▅▆▅▃▃█▆▆▄ ▃
wandb:     train/learning_rate ▄████▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:              train/loss ███████████████▇▆▅▄▃▃▂▂▂▂▁▁▂▂▁▁▁▁▁▁▂▁
wandb:     validation_accuracy ▁
wandb: 
wandb: Run summary:
wandb:            eval/accuracy 0.9344
wandb:                eval/loss 0.19723
wandb:             eval/runtime 122.9125
wandb:  eval/samples_per_second 40.679
wandb:    eval/steps_per_second 2.547
wandb:       

Step,Training Loss
25,0.694900
50,0.695700
75,0.695700
100,0.694400
125,0.695100
150,0.692200
175,0.694200
200,0.693400
225,0.690200
250,0.692600


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:           eval/accuracy ▁
wandb:               eval/loss ▁
wandb:            eval/runtime ▁
wandb: eval/samples_per_second ▁
wandb:   eval/steps_per_second ▁
wandb:            test/runtime ▁
wandb: test/samples_per_second ▁
wandb:   test/steps_per_second ▁
wandb:             train/epoch ▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
wandb:       train/global_step ▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇██████
wandb:         train/grad_norm ▃▃▂▂▁▁▂▄▃▃▂▄▃▂▇▅▆▆▆▄▅▆▅▃▄▄▅▅▄▅▃█▆▆▄▆▄
wandb:     train/learning_rate ▄████▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:              train/loss ██████████████▇▆▅▄▃▃▂▂▂▁▂▁▁▁▂▁▁▁▁▁▁▂▁
wandb:     validation_accuracy ▁
wandb: 
wandb: Run summary:
wandb:            eval/accuracy 0.9366
wandb:                eval/loss 0.19175
wandb:             eval/runtime 123.2109
wandb:  eval/samples_per_second 40.581
wandb:    eval/steps_per_second 2.54
wandb:        

Step,Training Loss
25,0.694900
50,0.695700
75,0.695700
100,0.694400
125,0.695200
150,0.692300
175,0.694300
200,0.693400
225,0.690300
250,0.692800


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:           eval/accuracy ▁
wandb:               eval/loss ▁
wandb:            eval/runtime ▁
wandb: eval/samples_per_second ▁
wandb:   eval/steps_per_second ▁
wandb:            test/runtime ▁
wandb: test/samples_per_second ▁
wandb:   test/steps_per_second ▁
wandb:             train/epoch ▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
wandb:       train/global_step ▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇██████
wandb:         train/grad_norm ▄▄▂▃▁▁▂▅▄▃▂▅▃▂█▄▇█▇▅▅▅▄▃▄▄▄▅▃▄▃▆▄▅▃▅▃
wandb:     train/learning_rate ▄████▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:              train/loss ███████████████▇▆▅▄▃▂▂▂▁▂▁▁▁▂▁▁▁▁▁▁▂▁
wandb:     validation_accuracy ▁
wandb: 
wandb: Run summary:
wandb:            eval/accuracy 0.937
wandb:                eval/loss 0.19373
wandb:             eval/runtime 123.1128
wandb:  eval/samples_per_second 40.613
wandb:    eval/steps_per_second 2.542
wandb:        

In [6]:
print(model)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DebertaV2ForSequenceClassification(
      (deberta): DebertaV2Model(
        (embeddings): DebertaV2Embeddings(
          (word_embeddings): Embedding(128100, 768, padding_idx=0)
          (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): DebertaV2Encoder(
          (layer): ModuleList(
            (0-11): 12 x DebertaV2Layer(
              (attention): DebertaV2Attention(
                (self): DisentangledSelfAttention(
                  (query_proj): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=16, bias=False)
      